In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import json
import joblib 

def get_data():
    # Simula la carga de un dataset
    data = load_iris()
    objeto1,objeto2 = data.data, data.target
    return objeto1, objeto2
X,y = get_data()

def preprocess_data(X):
    # Simula el preprocesamiento de los datos
    scaler = StandardScaler().fit(X)
    return scaler, scaler.transform(X)
z,Xz = preprocess_data(X)

def split_data(X,y):
    #Dividir la data
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5)
    return X_train, X_test, y_train, y_test
Xz_train, Xz_test, y_train, y_test = split_data(Xz,y)

def train_model(X_train, y_train):
    # Simula el entrenamiento de un modelo
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)
    return model
model = train_model(Xz_train, y_train)

def testing_model(model, X, y):
    # Simula la evaluación de un modelo
    predictions = model.predict(X)
    acc = accuracy_score(y, predictions)
    return {"accuracy": acc}

entrenamiento_metrics = testing_model(model, Xz_train, y_train)
test_metrics = testing_model(model, Xz_test, y_test)

if (test_metrics["accuracy"]>=0.95):
    with open("prod_config.json", "r") as f:
        config = json.load(f)
    batch_config = config["BATCH_PREDICTION"]
    sample_rate = batch_config["SAMPLE_RATE"]
    n_samples = int(len(Xz) * sample_rate)
    np.random.seed(42)
    sample_indices = np.random.choice(len(Xz), n_samples, replace=False)

    X_batch = Xz[sample_indices]
    predicciones_batch = model.predict(X_batch)

    feature_names = load_iris().feature_names
    X_df = pd.DataFrame(X_batch, columns=feature_names)
    X_df["prediction"] = predicciones_batch

    X_df.to_csv("batch_pred.csv",index= False,sep=";",decimal=",")
    joblib.dump(model, "modelo_entrenado.pkl")
    joblib.dump(z, "scaler_entrenado.pkl")
    